# One dimensional line search via estimation

A robust optimization technique that assumes the function to be optimized is continuous, but not differentiable, and noisy. The function is approximated by a parabola over the set of points that have been computed, assuming that the function is convex withn the range of points

Since JavaScript output is disabled in JupyterLab you will either need to run this in Jupyter notebooks (JupyterHub) to get plots, or check out the JupyterLab extensions at https://github.com/bokeh/jupyterlab_bokeh


In [16]:
# Example of optimization by bisection line search
import os, sys
import math
import bisect
import numpy as np

from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [4]:
# Initialization 
range_min, range_max = (-6,6)
# initial_guess = 0.5
# initial_step_size = 0.1
epsilon = 1.0e-6

# tuples of (x, f(x)), ordered by x.
search_grid = []

In [5]:
# A decidedly non-parabolic function with a global min 
def example_f(x):
    'Function over the range to minimize in one dim'
    sc = 2.60
    return sc*sc*math.exp((x-1.5)*sc) + sc*sc*math.exp(-(1.9 + x)*sc) +\
            10.0* math.sin(20* x) +\
            200000000.5*np.random.random_sample()

def add_f_to_grid(x, f=example_f):
    'Add points in sorted order to the sample'
    global search_grid
    # Binary search would be better
    found_k = [k for k in range(len(search_grid)) if abs(x-search_grid[k][0]) <= epsilon]
    if found_k == []:
        new_pt = (x, f(x))
        bisect.insort_left(search_grid, new_pt)
        return new_pt
    else:
        return search_grid[found_k[0]]
    
def init_grid(no_pts=3, shrinkage=9, f = example_f):
    global search_grid
    half_range = (range_max - range_min)/2
    grid_x = np.linspace(range_min + half_range/shrinkage, range_max - half_range/shrinkage, no_pts)
    search_grid = list(zip(grid_x, map(f, grid_x)))

In [6]:
# Use a quadratic regression to estimate the minimum of the function
# By centering the data around 0 the columns of the design matrix form an orthogonal basis function
def dm_sample(the_pt):
    'From an x,y tuple build a design matrix row'
    x = the_pt[0]
    return (1, x, x*x)

def points_design_matrix():
    'The quadratic regression design matrix'
    dm = np.empty((0,3),dtype='float')
    for row in search_grid:
        dm = np.vstack((dm, np.array(dm_sample(row), dtype='float')))
    return dm

In [7]:
def fit_parabola_to_sample():
    'Use conventional least squares to fit to the design matrix.'
    X = points_design_matrix()
    print('Design matrix:\n', X)
    y = [z[1] for z in search_grid]

    # The fit to the search grid points
    # For outputs, see https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.linalg.lstsq.html
    parabola_fit = np.linalg.lstsq(X,y, rcond=-1)
    # Return the coefficients (c, b a)
    return parabola_fit[0]

def fit_line_to_sample():
    'Use conventional least squares to fit to the design matrix.'
    X = points_design_matrix()
    X = X[:, (0,1)]
    print('Design matrix:\n', X)
    y = [z[1] for z in search_grid]
    # The fit to the search grid points
    # For outputs, see https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.linalg.lstsq.html
    linear_fit = np.linalg.lstsq(X,y, rcond=-1)
    # Return the coefficients (c, b a)
    return (linear_fit[0][0], linear_fit[0][1], 0)

# When the parabola fails, a linear or constant fit may show, by fitting better. 
def fit_constant():
    y = [z[1] for z in search_grid]
    if len(y) == 0:
        print("No samples found.", file=sys.stderr )
    y_est = np.mean(y)
    return  (y_est, 0, 0)

def eval_fit(pts, coeffs):
    'Compute the fitted values for the parabola fit'
    # pts - same format as search_grid
    c = coeffs[0]
    b = coeffs[1]
    a = coeffs[2]
    x_s = [z[0] for z in pts]
    y_s = [z[1] for z in pts]
    y_est = [a * x * x + b * x + c for x in x_s]
    # Compute estimated minimum
    if a != 0:
        print('Estimated min at: {}'.format(-0.5*b/a))
    # List the absolute value (L1) errors for the fit.
    errs = sum([abs(pt[1] - pt[0]) for pt in zip(y_est, y_s)])/len(y_s)
    print('residual mean abs error: {}'.format(errs))
    return [x_s, y_est]
    
def plot_search_grid(parabola_pts):
    p = figure(plot_width = 600, plot_height = 600, 
           title = 'Current points',
           x_axis_label = 'x', y_axis_label = 'f(x)')
    pts = list(zip(*search_grid))
    p.circle(pts[0], pts[1],  color = 'darkred', size=6)
    # Add a parabola approx.
    p.line(parabola_pts[0], parabola_pts[1], color = 'lightblue')
    show(p)

In [8]:
# Create some widely-spaced starting points, to broaden search over possible local optima. 
init_grid(10)
# Note that accuracy will improve if points distant from the estimated optimum are pruned 
#as more nearby points are added. 
search_grid

[(-5.333333333333333, 145259080.53384483),
 (-4.148148148148148, 52096524.74889924),
 (-2.962962962962963, 81197564.06685),
 (-1.7777777777777777, 154613599.380278),
 (-0.5925925925925926, 55981400.542742684),
 (0.5925925925925926, 86876902.53044033),
 (1.7777777777777777, 147164694.0399333),
 (2.962962962962963, 60585862.54459915),
 (4.148148148148148, 6526869.207701912),
 (5.333333333333333, 106223542.84778637)]

In [9]:
# This is how to create more points
for pt in np.linspace(0.25+range_min, range_max, 1):
   add_f_to_grid(pt)
search_grid

[(-5.75, 132974530.55988722),
 (-5.333333333333333, 145259080.53384483),
 (-4.148148148148148, 52096524.74889924),
 (-2.962962962962963, 81197564.06685),
 (-1.7777777777777777, 154613599.380278),
 (-0.5925925925925926, 55981400.542742684),
 (0.5925925925925926, 86876902.53044033),
 (1.7777777777777777, 147164694.0399333),
 (2.962962962962963, 60585862.54459915),
 (4.148148148148148, 6526869.207701912),
 (5.333333333333333, 106223542.84778637)]

In [10]:
# Return the coefficients of a quadratic regression
parabolic_fit = fit_parabola_to_sample()
print('Regression coefficients: {}'.format(parabolic_fit))
# Evaluate the fit at the sample points
print('>> Parabolic ', end='')
est_quadratic_pts = eval_fit(search_grid, parabolic_fit)

Design matrix:
 [[ 1.         -5.75       33.0625    ]
 [ 1.         -5.33333333 28.44444444]
 [ 1.         -4.14814815 17.20713306]
 [ 1.         -2.96296296  8.77914952]
 [ 1.         -1.77777778  3.16049383]
 [ 1.         -0.59259259  0.35116598]
 [ 1.          0.59259259  0.35116598]
 [ 1.          1.77777778  3.16049383]
 [ 1.          2.96296296  8.77914952]
 [ 1.          4.14814815 17.20713306]
 [ 1.          5.33333333 28.44444444]]
Regression coefficients: [87866034.03912215 -4464581.23810765   250443.351773  ]
>> Parabolic Estimated min at: 8.913355468414045
residual mean abs error: 36125774.08348278


In [11]:
# Also run a linear regression, and compare errors. 
linear_fit = fit_line_to_sample()
print('Regression coefficients: {}'.format(linear_fit))
# Evaluate the fit at the sample points
print('>> Linear ', end='')
est_linear_pts = eval_fit(search_grid, linear_fit)

Design matrix:
 [[ 1.         -5.75      ]
 [ 1.         -5.33333333]
 [ 1.         -4.14814815]
 [ 1.         -2.96296296]
 [ 1.         -1.77777778]
 [ 1.         -0.59259259]
 [ 1.          0.59259259]
 [ 1.          1.77777778]
 [ 1.          2.96296296]
 [ 1.          4.14814815]
 [ 1.          5.33333333]]
Regression coefficients: (91156510.81431144, -4657209.0513977, 0)
>> Linear residual mean abs error: 37031625.68084282


In [12]:
# OK also try a constant regression
coeff_const = fit_constant()
print('Regression coefficients: {}'.format(coeff_const))
print('>> Constant ', end='')
# Evaluate the fit at the sample points
est_const_pts = eval_fit(search_grid, coeff_const)

Regression coefficients: (93590961.00026937, 0, 0)
>> Constant residual mean abs error: 39687389.52006961


In [13]:
# Show both the search points and the best  fit
plot_search_grid(est_linear_pts)

In [14]:
# Show both the search points and the best parabolic fit
plot_search_grid(est_quadratic_pts)